# Data Polishing
The objective of this notebook is to give the data a final formatting and cleanup so it is more appropriate for presentation. Before starting, don't forget to setup your **virtual environment** with the necessary dependencies as specified in the `README.md` file.

## Imports
As usual, let's start by importing our libraries.

In [2]:
import csv
import json
import pprint

Following, let's load our current **training dataset** and take a look at the data.

In [14]:
with open('../RawData/modified_media_items.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    current_media_data = []
    for row in csv_reader:
        record_id = row[0]
        media_id = row[1]
        media_title = row[2]
        current_media_data.append([record_id, media_id, media_title])
        
# Take the headers out of the data
current_media_data = current_media_data[1:]
        
pprint.pprint(current_media_data[1000:1020])
print("Media Items:", len(current_media_data))

[['1001', '1436', 'Mr. Jones'],
 ['1002', '1179', 'Man of the House'],
 ['1003', '354', 'Wedding Singer, The'],
 ['1004', '1629', 'Nico Icon'],
 ['1005', '839', 'Loch Ness'],
 ['1006', '216', 'When Harry Met Sally...'],
 ['1007', '956', "Nobody's Fool"],
 ['1008', '1035', 'Cool Runnings'],
 ['1009', '1603', 'Angela'],
 ['1010', '573', 'Body Snatchers'],
 ['1011', '667', 'Audrey Rose'],
 ['1012', '1233', 'N\\xc3\\xa9nette et Boni'],
 ['1013', '67', 'Ace Ventura: Pet Detective'],
 ['1014', '1387', 'Fall'],
 ['1015', '924', 'White Squall'],
 ['1016', '1025', 'Fire Down Below'],
 ['1017', '1545', 'Frankie Starlight'],
 ['1018', '1524', 'Kaspar Hauser'],
 ['1019', '1112', 'Cobb'],
 ['1020', '414', 'My Favorite Year']]
Media Items: 1682


There is not too much to fix on the Anime titles format, but when we get to the movie titles we can see that some of them have *The* at the end, making it hard to read. This might also present a problem for the Front-end team when they make calls to the Recommender System. Let's check what kind of discrepancies we have and how many times they occur.

In [20]:
discrepancies = {}

for media in current_media_data:
    index = media[2].find(", ") 
    if index > -1:
        key = media[2][index:]
        if key in discrepancies.keys():
            discrepancies[key].append(media[1])
        else:
            discrepancies[key] = [media[1]]
            
pprint.pprint(discrepancies)

{', A': ['1493',
         '1094',
         '179',
         '507',
         '686',
         '1063',
         '1665',
         '814',
         '1075',
         '1602',
         '964',
         '560',
         '1181',
         '553',
         '886',
         '412',
         '189',
         '1576',
         '1671',
         '1292',
         '1310',
         '1219'],
 ', An': ['602', '436', '1227'],
 ', Aruiwa Sekai ga Hajimaru Seisen (Zoku-hen)': ['454'],
 ', Chouju Ryouri Taiketsu!!': ['1060'],
 ', Corrina': ['731'],
 ', Das': ['515'],
 ', Fukenkou yo! (Kari)': ['80'],
 ', Good Night (Re:Dialed)': ['1410'],
 ', Guido': ['995'],
 ', Hana no 17-sai': ['290'],
 ', Haruka': ['625'],
 ', Iwarenaki Jaryuu Nintei': ['586'],
 ', Konkon, Koi Iroha.: Inari, Konkon, Semishigure.': ['1071'],
 ", L'": ['1535'],
 ', La': ['1623', '1578', '296'],
 ', La (Double Life of Veronique, The)': ['1005'],
 ', La (Life is Rosey)': ['1569'],
 ', La)': ['1097', '730'],
 ', Le': ['1382'],
 ', Les': ['543'],
 ', Les)

Now we can see what discrepancies are the most frequent and where to find them exactly with the matching `media_id`s. Let's start by fixing the most common ones.

In [21]:
for media in current_media_data:
    if media[1] in discrepancies[", A"]:
        media[2] = "A " + media[2][:-3] 
    if media[1] in discrepancies[", The"]:
        media[2] = "The " + media[2][:-5] 

We can now procede to save our data in a new CSV file.

In [25]:
with open('../RawData/clean_media_items.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["record_id", "media_id", "media_title"])

    for media in current_media_data:
        csv_writer.writerow([media[0], media[1], media[2]])

There is still some processing to do in our data, but since there are not so many cases and they are kind of special, we will do this manually on the CSV file. After that we will reload the data again.

In [10]:
with open('../RawData/clean_media_items.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    clean_media_items = []
    for row in csv_reader:
        record_id = row[0]
        media_id = row[1]
        media_title = row[2]
        clean_media_items.append([record_id, media_id, media_title])
        
clean_media_items = clean_media_items[1:]
        
pprint.pprint(clean_media_items[:20])
print("Media Items:", len(clean_media_items))

[['1', '1198', 'Classroom Crisis'],
 ['2', '939', 'Highlander: The Search for Vengeance'],
 ['3', '1141', 'Chuuken Mochi Shiba'],
 ['4', '1601', 'Renou Xueyuan'],
 ['5', '922', 'Jashin-chan Dropkick Episode 12'],
 ['6', '1190', 'Ketsuekigata-kun! 3'],
 ['7', '230', 'Sanguo Yanyi 2nd Season: Zhulu Zhongyuan'],
 ['8', '607', 'Danball Senki'],
 ['9', '1435', '8-gatsu no Symphony: Shibuya 2002-2003'],
 ['10', '1557', 'Koisuru Boukun'],
 ['11', '914', 'Chang Jian Fengyun 2'],
 ['12', '1398', 'Mahoutsukai no Yome'],
 ['13', '365', 'New Big Head Son and Little Head Dad Season 3'],
 ['14', '1414', 'SINBAD'],
 ['15', '1078', 'Message Song'],
 ['16', '505', 'Oren no yurai'],
 ['17', '794', 'Yakimochi Caprice'],
 ['18', '771', 'Kanojo to Kanojo no Neko: Everything Flows'],
 ['19', '1133', 'Garakuta-doori no Stain'],
 ['20', '1127', 'Huanbao Tegong Dui']]
Media Items: 1682


Let's also load the ratings for the final processing.

In [11]:
with open('../RawData/modified_ratings.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    modified_ratings = []
    for row in csv_reader:
        record_id = row[0]
        media_id = row[1]
        media_title = row[2]
        user_id = row[3]
        user_rating = row[4]
        modified_ratings.append([record_id, media_id, media_title, user_id, user_rating])
        
modified_ratings = modified_ratings[1:]
        
pprint.pprint(modified_ratings[:20])
print("Ratings:", len(modified_ratings))

[['1', '357', 'Sifan', '138', '4.0'],
 ['2', '709', 'Strictly Ballroom', '92', '2.0'],
 ['3', '412', 'Very Brady Sequel, A', '301', '4.0'],
 ['4', '56', 'Pulp Fiction', '60', '4.0'],
 ['5', '895', 'Kobo-chan: Matsuri ga Ippai!', '197', '3.0'],
 ['6', '325', 'Crash', '601', '4.0'],
 ['7', '95', 'Zhun Xing', '710', '3.0'],
 ['8', '92', 'True Romance', '833', '2.0'],
 ['9', '425', 'Bob Roberts', '916', '5.0'],
 ['10', '271', 'Starship Troopers', '940', '2.0'],
 ['11', '355', 'Sphere', '611', '1.0'],
 ['12', '712', 'Baosheng Dadi Zhi Qi Er Duo Bao', '707', '3.0'],
 ['13', '825', 'Arrival, The', '699', '3.0'],
 ['14', '240', 'Zhu Zhu Xia: Jing Qiu Xiao Yingxiong', '16', '4.0'],
 ['15', '1150', 'Last Dance', '314', '4.0'],
 ['16', '684', 'In the Line of Fire', '217', '5.0'],
 ['17', '124', 'Lone Star', '276', '5.0'],
 ['18', '294', 'Karakuri Circus (TV)', '510', '3.0'],
 ['19', '265', 'Haitoku no Kyoukai', '757', '3.0'],
 ['20', '465', 'Chan Shuo A Kuan', '881', '3.0']]
Ratings: 100000


Now, let's replace one more time the titles in the ratings with the new data.

In [14]:
for rating in modified_ratings:
    for media in clean_media_items:
        if media[1] == rating[1]:
            rating[2] = media[2]
            
pprint.pprint(modified_ratings[:20])
print("Ratings:", len(modified_ratings))

[['1', '357', 'Sifan', '138', '4.0'],
 ['2', '709', 'Strictly Ballroom', '92', '2.0'],
 ['3', '412', 'A Very Brady Sequel', '301', '4.0'],
 ['4', '56', 'Pulp Fiction', '60', '4.0'],
 ['5', '895', 'Kobo-chan: Matsuri ga Ippai!', '197', '3.0'],
 ['6', '325', 'Crash', '601', '4.0'],
 ['7', '95', 'Zhun Xing', '710', '3.0'],
 ['8', '92', 'True Romance', '833', '2.0'],
 ['9', '425', 'Bob Roberts', '916', '5.0'],
 ['10', '271', 'Starship Troopers', '940', '2.0'],
 ['11', '355', 'Sphere', '611', '1.0'],
 ['12', '712', 'Baosheng Dadi Zhi Qi Er Duo Bao', '707', '3.0'],
 ['13', '825', 'The Arrival', '699', '3.0'],
 ['14', '240', 'Zhu Zhu Xia: Jing Qiu Xiao Yingxiong', '16', '4.0'],
 ['15', '1150', 'Last Dance', '314', '4.0'],
 ['16', '684', 'In the Line of Fire', '217', '5.0'],
 ['17', '124', 'Lone Star', '276', '5.0'],
 ['18', '294', 'Karakuri Circus (TV)', '510', '3.0'],
 ['19', '265', 'Haitoku no Kyoukai', '757', '3.0'],
 ['20', '465', 'Chan Shuo A Kuan', '881', '3.0']]
Ratings: 100000


Finally, let's save our progress in a CSV file.

In [15]:
with open('../RawData/clean_ratings.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["record_id", "media_id", "media_title", "user_id", "user_rating"])

    for rating in modified_ratings:
        csv_writer.writerow([rating[0], rating[1], rating[2], rating[3], rating[4]])

Now we can proceed to create the new Datasets with the polished data.